In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

if os.getcwd().endswith("notebooks"):
    os.chdir("../..")

from ptseries.tbi import create_tbi
from ptseries.algorithms.binary_solvers import BinaryBosonicSolver
from ptseries.common.logger import Logger
from ptseries.common.set_seed import set_seed

set_seed(0)

# The Workshop Optimisation Problem
<center><img src="../figures/Garage_Img.webp" width="800"><center>

In this notebook, we will go over how to use the BBS algorithm to solve large *knapsack problems* posed as a garage workshop trying to determine which items to order. The notebook will cover the following:
- A motivated example of a knasack problem and setup of a binary optimisation task
- Setup of the BBS algorithm to solve a simple example of the problem
- Using the BBS algorithm to solve a more complicated knapsack problem
- Adding complicated constraints to the problem

We will assume basic knowledge of how the setup of the BBS algorithm works, and basic familiarity with knapsack problems. If you are still unsure of this, see the `optimisation_intro.ipynb` notebook first.

## A Simple Knapsack Problem
Knapsack problems are tasks where each item we take gives us a certain amount of value, but each item has a given weight, and we have a maximum weight capacity we can't go over. We want to pick the best choice of items to maximise value. In this notebook, we will consider a garage that is deciding which items to order for its stores. Each item ordered provides a certain amount of profit to the garage (when sold), but takes up space in the garage's storage space. We want to find the best choice of items to order to get the most expecteed profit.

Let's consider a small garage that has a limited amount of storage space, say $100 \textrm{m}^3$. They can order cars, wheels or engines and they need a certain amount of space for each of these. We assume that whatever they have in stock, they will sell and this gives them a certain profit which we define below.

In [ ]:
# Amount of storage space the Garage has (say, cubic metres)
storage_space = 100

# The items the Garage can order
items = ["Car", "Wheel", "Engine"]

# The profit they get from each instance of a given item they have in stock
profits = {"Car": 20000, "Wheel": 80, "Engine": 1000}

# The amount of space each item takes up
sizes = {"Car": 35, "Wheel": 1, "Engine": 3}

# Converts the utilities and sizes to lists in the 'correct' order as the items
profits = [profits[x] for x in items]
sizes = [sizes[x] for x in items]

In the `optimisation_intro.ipynb` notebook, we considered a simple knapsack problem endocing into a binary optimisation task where every single bit in our bit string represented whether an item was taken or not. This is quite inefficient as we need bit strings of length $m$ to solve a task with $m$ items at most to be taken. In terms of hardware, this represents simulating a time bin interferometer (PT Series) with $m/2$ photons and $m$ modes. It will quickly become hard to solve problems with lots of items.

For this task, we want to be able to consider many copies of any given item being ordered. As such, we consider allocating several bits to each item, and encoding the number of copies of that item we take as a number in binary. Say we want to allow at most 15 of each item to be ordered. We can encode numbers from 0 to 15 as a bit string of length 4. So, for our three items, we can take a bit string of length (4 * 3 ) = 12, and break it up into 3 length 4 substrings, each of which encodes how many of a certain item we are ordering. For our algorithm to work well, we'd like it to be the case that if we change one bit in our string (flip a 0 to a 1), then there isn't a big change to the order that we are encoding. To achieve this, we will use 'GrayCode', which is an ordering of binary numbers such that two numbers that are next to each other only differ by one bit when encoded in binary.

In [ ]:
# The 'n_bits' variable stores the number of bits we allocate in our bit string to counting the number of each item
n_bits = 4


# Converts a binary string to GrayCode (a different ordering for binary numbers)
def binary_to_gray(bit_string):
    n = int(bit_string, 2)
    return n ^ (n >> 1)


# We define a function that converts a binary string to the number of orders we take of each item
def binary_to_orders(bit_string, n_items, n_bits):
    # Converts numpy arrays to lists
    if type(binary_to_orders) == "numpy.ndarray":
        bit_string = bit_string.tolist()

    # Divides the binary string into blocks encoding the number of orders of each item
    order_amounts = [bit_string[n_bits * i : n_bits * (i + 1)] for i in range(n_items)]

    # Converts the string for each item into an order number
    order_amounts = [binary_to_gray("".join(str(i) for i in x)) for x in order_amounts]

    return order_amounts


Let's test this out. We can define a bit string of length 12 ( number of bits per item (4) $\times$ number of items (3)) and see how this gets converted to the number of each item to order. Remember, we're using the GrayCode ordering of the binary numbers, so the string '1111' won't encode '15' anymore, but '8'.

In [ ]:
# We define a bit string, and then see how this gets turned into a number of orders
bit_str = [0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]
print(binary_to_orders(bit_str, 3, 4))

Now its time to encode a cost function. Given a list of item order numbers, we calculate the total profit these orders give us. If the total space taken up by the orders goes over the maximum capacity, we add a penalty term (the same cost function formulation we've seen in the introductory notebook). At the end of all of this, the function returns the negative of this calculated value. This is because BBS is a minimising algorith, it acts to find the solution which gets the smallest possible value of the cost function.

In [ ]:
def cost_fn_fact(profits, sizes, storage_space, penalty):
    def cost_fn(bit_string):
        bit_string = bit_string.astype(int)

        # Calculates the number of bits used for each item
        n_bits = int(len(bit_string) / len(profits))

        # Gets the number of orders from the bit string
        order_numbers = np.array(binary_to_orders(bit_string, len(profits), n_bits))

        # Calculates total profit
        profit_total = np.dot(order_numbers, np.array(profits))

        # Calculates total space taken
        space_total = np.dot(order_numbers, np.array(sizes))

        # Penalises the profit heavily if more space taken by items than storage space
        if space_total > storage_space:
            profit_total += -penalty

        return (-1) * profit_total / 10**4

    return cost_fn

### Run the Optimisation Algorithm
With all of our variables and cost function defined, we can now run the BBS optimisation algorithm using a time bin interferometer simulator which tries to find a solution which has the smallest value from the cost function. The bit strings we are using here have length 12, which means we need to simulate a PT Series device that has 6 photons and 12 modes. This is comfortably within the specifications of the PT-2. Below, we draw a diagram of the linear optical circuit that we're simulating.

In [ ]:
# We draw the TBI configuration which we will be using for the optimisation
tbi = create_tbi(n_loops=1)

# Calculate the number of photons in the circuit (half the number of modes)
n_photons = int(len(profits) * n_bits / 2)

# Calculating the input state the tbi is using
input_state = (
    1,
    0,
) * n_photons

tbi.draw(input_state=input_state)

Now, we initialise the binary bosonic solver algorithm and train it in the usual way.

In [ ]:
logger = Logger(log_dir=None)

# Defining the cost function we use
profit_cost_fn = cost_fn_fact(profits, sizes, storage_space, 10**6)

# Defines the length of a possible solution bit string
bit_str_length = len(items) * n_bits

# Initialises the algorithm
bbs = BinaryBosonicSolver(bit_str_length, profit_cost_fn, entropy_penalty=0)

# Trains the algorithm
bbs.train(learning_rate=1e-3, updates=150, print_frequency=30, logger=logger)
print("Training finished")

opt_sol = np.array(bbs.config_min_encountered)
opt_sol = opt_sol.astype(int)

Using the `logger` (see `optimisation_intro.ipynb` for details), we plot the losses the model gets during training. The *loss* values represent a measure of how good the solutions that the algorithm is finding are. The lower the loss, the better the solutions the algorithm is finding are. The losses decreasing means that the solution we've found is getting better as we go through update steps.

In [ ]:
# Plot the values for the loss as we go through update steps
def plot_training(logger):
    x_values_qubo = list(map(int, logger.logs["energy_avg"].keys()))
    energies_avg_qubo = list(logger.logs["energy_avg"].values())
    plt.plot(x_values_qubo, energies_avg_qubo)
    plt.xlabel("Updates")
    plt.ylabel("Loss")
    plt.show()


plot_training(logger)

We output the best solution found by the algorithm, and format our findings.

In [ ]:
# Calculates the number of each item we have taken in our optimal solution
opt_orders = binary_to_orders(opt_sol, len(items), n_bits)

# Calculates the total profit and space taken by the optimum solution
opt_profit = np.dot(np.array(opt_orders), np.array(profits))
opt_space = np.dot(np.array(opt_orders), np.array(sizes))

# Formats an output of the optimal solution
print("The best solution found is to take :")
for index, times_taken in enumerate(opt_orders):
    if times_taken > 0:
        if times_taken > 1:
            suffix = "s"
        else:
            suffix = ""
        print(str(times_taken) + " " + items[index] + suffix)

print(
    f"This gets us a profit of ${opt_profit} while taking up {opt_space} m^3 of space"
)

We've got our solution! The algorithm will give us the solution of taking 2 cars and 10 engines. It's important to note that sometimes, running the algorithm may not give this exact solution. This is because BBS uses a simulated PT Series device which is probabilistic in how it acts. For any two runs of the algorithm, how the simulated PT Series acts will be different.

In this simple example it's also very easy to check that this is the exact unique solution also. We order as many cars as possible, as they give us the best profit per metre cubed of space being taken up, and then fill the rest of the space with the second most profit/space efficient item, the engines. However, we do run into the issue that we haven't ordered any wheels. Wheels don't have as good of a profit/space ratio as the other two items, however to run a good garage we need to have a variety of goods in stock to meet the customers needs, not only those with the best margins. We need a more realistic cost function, which we define in the next section.

## A Harder Problem - The Big Garage
We've tackled a very simple problem where it's easy to see what the optimal solution is. Now we're going to go onto something more complicated, but we will use the exact same steps as before to use the BBS algorithm to find a solution.
 
 Now lets say we have a bigger garage, that is trying to make sure it has enough items to meet customer demand. The garage has all of its sales data for the previous months, and they use this to build a functions that gives the expected amount of profit having a certain amount of an item will give. The Garage can order 100 cars, but it's very unlikely they'll sell all 100 in a month, so the expected profit you get from each next item you have in stock diminishes. To be exact, the garage has calculated a minimum number of sales for each item that they know they will get, and then a maximum number of sales of each item after which they are certain they'll get no more sales. The garage has used the previous order statistics to create a function that maps the number of items they have to the expected revenue they'll get from sales in that month from the items with these constraints.

We define the function the garage uses to calculate the expected profit it gets from instances of an item.

In [ ]:
# Defines a function that returns the expected revenue generated from having a certain number of an item, where 'sale price'
# is how much the item costs to buy, 'min' is the minimum number of items the garage is sure will sell, and 'max' is the
# maximum number of items that could possibly be sold.


def expected_profit(n_items, min, max, sale_price):
    A = 0
    B = 0
    C = 0

    B = np.pi / (2 * max)

    A = sale_price / (B * np.cos(B * min))

    C = sale_price * min - A * np.sin(B * min)

    if n_items < min:
        return n_items * sale_price

    elif n_items < max:
        return A * np.sin(B * n_items) + C

    else:
        return A * np.sin(B * max) + C


For example, say that the garage found that in any given month at least 5 cars are sold, but no month ever has more than 20 cars being sold, and each car sells for $20,000. Then, the expected profit that the garage has calculated having a given number of cars in stock gives, is given by the following function we plot below.

In [ ]:
# Amount of cars we're certain to sell
min = 5

# Amount of cars past which we know we will sell none
max = 20

# Price of each car (in units of $10,000)
price = 2

n_cars = np.arange(0, 31)

expected_profits = [expected_profit(x, min, max, price) for x in n_cars]

plt.plot(expected_profits)
plt.title("Expected Profit with Number of Cars Ordered")
plt.xlabel("Number of Cars")
plt.ylabel("Expected Profit")
plt.axvline(x=min, linestyle="--", color="red", label="certain min. demand")
plt.axvline(x=max, linestyle="--", color="blue", label="max. possible demand")
plt.legend()
plt.show()

The expected profit we get from each subsequent car gets smaller and smaller until ordering extra cars doesn't give us any added value at all. Now, we can begin to pose the full problem we're trying to solve. The garage is trying to figure out how much of each item to order in for the next month. We have a list of items, with each item taking up a certain amount of space, and having an associated cost. By looking back at past order data, the garage has calculated the amount of sales they are sure they will get for each item, and also the amount past which they are sure they will not sell any more. The garage as before has a limited amount of space. Furthermore, to make sure consumer demand is being met well, the garage wants to make sure that, for all sales it's sure it will get in the month, it has an item in stock to meet it (irregardless of if this is the most profitable option).

We're going to start posing this problem by defining the proper variables in python, exactly as before.

In [ ]:
# Amount of storage space the Garage has (cubic metres)
storage_space = 700

# The items the Garage can order
items = ["Car", "Wheel", "Engine", "Truck", "Battery", "Child Seat"]

# The price of each item
prices = {
    "Car": 20000,
    "Wheel": 400,
    "Engine": 1000,
    "Truck": 35000,
    "Battery": 2000,
    "Child Seat": 7000,
}

# The amount of space each item takes up
sizes = {
    "Car": 35,
    "Wheel": 1,
    "Engine": 3,
    "Truck": 70,
    "Battery": 8,
    "Child Seat": 20,
}

# The minimum number of each item thats sure to be sold
min_sales = {
    "Car": 4,
    "Wheel": 10,
    "Engine": 3,
    "Truck": 2,
    "Battery": 1,
    "Child Seat": 8,
}

# The maximum number of each item past which it's certain none will be sold
max_sales = {
    "Car": 8,
    "Wheel": 30,
    "Engine": 7,
    "Truck": 4,
    "Battery": 4,
    "Child Seat": 12,
}

# Converts sizes to lists in the same order as the items
sizes = [sizes[x] for x in items]

# The number of bits we allocate to decide order number for each item
n_bits = 5  # With 5, we can order at most 31 of each item

We have our converter of binary strings to order counts defined from earlier, so all that is left to do is to define the cost function we'll be using.

In [ ]:
def cost_fn_fact(prices, sizes, storage_space, min_sales, max_sales):
    def cost_fn(bit_string):
        bit_string = bit_string.astype(int)

        # The penalty we incur if we break a constraint
        penalty = 10**6

        # Calculates the number of bits used for each item
        n_bits = int(len(bit_string) / len(prices))

        # Gets the number of orders from the bit string
        order_numbers = np.array(binary_to_orders(bit_string, len(prices), n_bits))

        # Calculates total profit we get from the number of items
        profit_total = 0

        for i in range(len(order_numbers)):
            order_num = order_numbers[i]
            profit = expected_profit(
                order_num, min_sales[items[i]], max_sales[items[i]], prices[items[i]]
            )
            profit_total += profit

        # Calculates total space taken
        space_total = np.dot(order_numbers, np.array(sizes))

        # Penalises the profit heavily if more space taken by items than storage space
        # In fact, we scale the penalty such that if we are way above the storage space limit, we give a bigger penalty
        if space_total > storage_space:
            profit_total += (-1) * (10**6 + (space_total - storage_space) * 5 * 10**4)

        # Penalises the profit heavily if we don't order at least as many of each item as we know will be sold
        # In fact, we scale the penalty such that if we have far less items in than the minimum, we get a bigger penalty
        for i in range(len(order_numbers)):
            order_num = order_numbers[i]
            if order_num < min_sales[items[i]]:
                profit_total += (-1) * penalty
                profit_total += (-1) * (10**5) * (min_sales[items[i]] - order_num)

        return (-1) * profit_total / 10**6

    return cost_fn

Now, we can proceed exactly as before. We have 6 items and 5 bits per item which means the bit strings which we are considering have total length 30. This means we need to use linear optical circuits with 15 photons and 30 modes. This is within the capacity of the specifications of the PT-2!

With the length of the bit strings we are considering here being 30, there are $2^{30} = 1073741824$ possible different item order numbers that we need to choose from. We can see how very quickly these problems become hard to solve. It would be unreasonable to consider every single possible solution seperately.

We use the BBS algorithm to get towards a solution. Note that now, as we are considering bigger bit strings, we have to simulate the action of a bigger PT Series device. This means that the algorithm now runs far slower on our classical computer than the previous one, as simulating a bigger PT Series device quickly becomes very hard for a classical computer.

We will plot out the losses as they get minimised by the BBS algorithm during training.

In [ ]:
logger = Logger(log_dir=None)
profit_cost_fn = cost_fn_fact(prices, sizes, storage_space, min_sales, max_sales)

# Defines the length of a possible solution bit string
bit_str_length = len(items) * n_bits

bbs = BinaryBosonicSolver(
    bit_str_length, profit_cost_fn, tbi_params={"tbi_type": "single-loop"}
)

bbs.train(learning_rate=1e-2, updates=150, print_frequency=10, logger=logger)
print("Training finished")

opt_sol = np.array(bbs.config_min_encountered)
opt_sol = opt_sol.astype(int)

In [ ]:
plot_training(logger)

Again, we get the best solution the algorithm has come across and format it as an output.

In [ ]:
# Calculates the number of each item we have taken in our optimal solution
opt_orders = binary_to_orders(opt_sol, len(items), n_bits)

# Calculates the total profit and space taken by the optimum solution
opt_profit = (-1) * int(round(profit_cost_fn(opt_sol) * 10**6, 0))
opt_space = np.dot(np.array(opt_orders), np.array(sizes))

# Formats an output of the optimal solution
print("The best solution found is to take :")
for index, times_taken in enumerate(opt_orders):
    if items[index] != "Battery":
        if times_taken > 0:
            if times_taken > 1:
                suffix = "s"
            else:
                suffix = ""
            print(str(times_taken) + " " + items[index] + suffix)

    elif items[index] == "Battery":
        if times_taken > 0:
            if times_taken > 1:
                print(str(times_taken) + " Batteries")
            else:
                print(str(times_taken) + " Battery")

print(
    "This gets us a profit of $"
    + str(opt_profit)
    + " while taking up "
    + str(opt_space)
    + "m^3 of space"
)

And we've done it! We've got a good solution to an interesting problem which would be very hard to optimise over by hand. It's important to note that the solution you'll generate here likely won't be completely optimal, it will however probably be quite close. For the garage, it's important to get a very good solution quickly rather than a perfect solution after a very long time. With how big the number of solutions is, searching for the perfect solution takes a very long amount of time. To improve the solution we find, consider increasing the number of updates higher. This will take longer to run, but should reach a better solution in the end.

One thing that the solution will likely have is that it may not use all 700 m^3 of space (although it will probably be close). This is because the algorithm is penalised heavily if it goes above the maximum storage limit, so it will leave a bit of room to make sure this penalty isn't taken. It is certainly possible to account for this and add a penalty if a solution *doesn't* use all of the space, to motivate the algorithm to do so.

### Adding Sales Promotions

Now, say the garage wants to run a promotion to try to boost profits. The garage starts offering cars at a cheaper price, and knows that doing this will boost demand for cars. We want to see how this will affect the optimal order values, and if doing this will boost revenue for the month. We'll change the car price and car demand values to see how this affects our solution.

In [ ]:
prices["Car"] = prices["Car"] * 0.8  # We take 20% off the price of cars

min_sales["Car"] = (
    min_sales["Car"] + 2
)  # We think that doing this will increase car demand for the month by 2
max_sales["Car"] = max_sales["Car"] + 2

We'll now run our training algorithm exactly as before, with the promotion now being added. Again, we'll plot out the losses and see how they evolve during the course of the algorithm.

In [ ]:
logger = Logger(log_dir=None)

# Defining the cost function we use (as usual)
profit_cost_fn = cost_fn_fact(prices, sizes, storage_space, min_sales, max_sales)

# Defines the length of a possible solution bit string
bit_str_length = len(items) * n_bits

bbs = BinaryBosonicSolver(
    bit_str_length, profit_cost_fn, tbi_params={"tbi_type": "single-loop"}
)

bbs.train(learning_rate=1e-2, updates=150, print_frequency=10, logger=logger)
print("Training finished")

opt_sol = np.array(bbs.config_min_encountered)
opt_sol = opt_sol.astype(int)

In [ ]:
plot_training(logger)

In [ ]:
# Calculates the number of each item we have taken in our optimal solution
opt_orders = binary_to_orders(opt_sol, len(items), n_bits)

# Calculates the total profit and space taken by the optimum solution
opt_profit = (-1) * int(round(profit_cost_fn(opt_sol) * 10**6, 0))
opt_space = np.dot(np.array(opt_orders), np.array(sizes))

# Formats an output of the optimal solution
print("The best solution found is to take :")
for index, times_taken in enumerate(opt_orders):
    if items[index] != "Battery":
        if times_taken > 0:
            if times_taken > 1:
                suffix = "s"
            else:
                suffix = ""
            print(str(times_taken) + " " + items[index] + suffix)

    elif items[index] == "Battery":
        if times_taken > 0:
            if times_taken > 1:
                print(str(times_taken) + " Batteries")
            else:
                print(str(times_taken) + " Battery")

print(
    "This gets us a profit of $"
    + str(opt_profit)
    + " while taking up "
    + str(opt_space)
    + "m^3 of space"
)

And there we have it! The solution we've found here gets us less revenue for the month than the one we got without the promotion on car prices. The garage can conclude that it might be best to not run the promotion. We can easily try out adding different promotions in exactly the same way : by changing the price and demand parameters.